Place this code piece next to the .../4_power/home_assignment.ipynb to run

In [172]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from gql import gql, Client
from gql.transport.aiohttp import AIOHTTPTransport

from etherscan import Etherscan

import json
import time
import os
import random
import math

import scipy.stats as st


print(pd.__version__)
print(np.__version__)
print(requests.__version__)
print(plt.matplotlib.__version__)


1.4.3
1.21.5
2.28.1
3.5.2


In [173]:
# https://martin-thoma.com/configuration-files-in-python/

import json

with open("../3_api/.private/keys.json") as keys_file:
    KEYS = json.load(keys_file)

# Note: don't print the key, or if you do, delete the cell's output
# (cell outputs are saved and can be sent to Github).

In [174]:
## SNAPSHOT
###########

SNAPSHOT_ENDPOINT = "https://hub.snapshot.org/graphql"

snapshot = Client(
    transport=AIOHTTPTransport(url=SNAPSHOT_ENDPOINT)
)


def snapshot_rest(query, params=None):

    response = requests.post(SNAPSHOT_ENDPOINT,
                            headers={                      
                                'accept': 'application/json'
                            },
                            params={
                                'query': query
                            })

    print(response)
    return response.json()['data']

In [175]:
def pd_read_json(file):
    ## Prevents Value too big Error.
    with open(file) as f:
        df = json.load(f)
    df = pd.DataFrame(df)
    return df

In [176]:
def get_query(filename, do_gql=False):
    with open("gql_queries/" + filename.replace(".gql", "") + ".gql") as f:
        query = f.read()
        if do_gql: query = gql(query)
    return query


In [177]:
async def gql_all(query, field, first=1000, skip=None, initial_list=None, 
                  counter = True, limit=None, save=None, save_interval=10, clear_on_save = False, append=True, rest=False, data_dir="data", save_counter = 1, vars=None):

    ## The returned value and the varible used to accumulate results.
    out = []

    ## Utility function to save intermediate and final results.
    def save_json():

        # Pandas has problem load pure json saves.
        # Hence we create a pandas Dataframe and save it.
        # nonlocal append
        # flag = "a" if append else "w"
        # with open("data/" + save, flag) as f:
        #     json.dump(out, f)
        #     print("Saved.")

        nonlocal out
        df = pd.DataFrame(out)

        if clear_on_save:
            
            nonlocal save_counter
            
            sv = str(save_counter)
            sv = sv.zfill(5)
            save_counter += 1

            filename = save.replace('.json', '_' + sv + '.json')
            
            out = []
            out_str = "Saved and cleared."
        else:
            filename = save
            out_str = "Saved."
        
        df.to_json(data_dir + "/" + filename, orient="records")
        print(out_str)

        
    ## Load initial list.
    ## If no skip is provided, then skip is set to the length of
    ## the initial list, otherwise we use the user-specified value
    if initial_list:
        out = initial_list
        if skip is None:
            skip = len(out)
    elif skip is None:
        skip = 0

    ## Make a GQL query object, if necessary.
    if not rest and type(query) == str:
        query = gql(query)
        

    my_counter = 0
    fetch = True
    try:
        while fetch:
            
            my_counter += 1
            if limit and my_counter > limit:
                print('**Limit reached: ', limit)
                fetch = False
                continue

            if rest:

                # Building query manually.
                q = query.replace("($first: Int!, $skip: Int!)", "")
                q = q.replace("$first", str(first))
                q = q.replace("$skip", str(skip))
                # print(q)

                ## Optional additional variables.
                if vars:
                    for v in vars:
                        q = q.replace("$" + v, str(vars[v]))

                res = snapshot_rest(q)
                
            else:
                
                _vars = {"first": first, "skip": skip}
                
                ## Optional additional variables.
                if vars:
                    _vars = _vars | vars 

                res = await snapshot.execute_async(query, variable_values=_vars)
            
            if not res[field]:
                print('**I am done fetching!**')
                fetch = False
            else:
                out.extend(res[field])
                skip += first
                if counter: print(my_counter, len(out))

                if save and my_counter % save_interval == 0:
                    save_json()

        if save and my_counter % save_interval != 0:
            save_json()

    except Exception as e:
        print(str(e))
        print("**An error occurred, exiting early.**")
        if save: save_json()
    
    return out

In [178]:
def pd_read_dir(dir, blacklist=None, whitelist=None, ext=('.json')):
    dir_df = pd.DataFrame()
    

    for file in os.listdir(dir):
        if blacklist and file in blacklist:
            continue
        if whitelist and file not in whitelist:
            continue

        if file.endswith(ext):
            tmp_df = pd_read_json(dir + '/' + file)
            dir_df = pd.concat([dir_df, tmp_df])
    
    return dir_df

In [179]:
ENDPOINT = 'https://hub.snapshot.org/graphql'

def snapshot_api(query, params=None):

    response = requests.post(ENDPOINT,
                            headers={                      
                                'accept': 'application/json'
                            },
                            params={
                                'query': query
                            })

    print(response)
    return response.json()['data']

Experiment with one:
https://snapshot.org/#/ens.eth/proposal/0x4a1aedbd9d22295f358dc4028b5a3f0a602bb5f1089dabdc2b63bf2bcce45834

In [221]:
query = """
query {
  proposals (
    where: {first: 10000, skip:10000}
  ) {
    id
    author
    created
    title
    state
    choices
    scores
    scores_by_strategy
    scores_state
    scores_total
    scores_updated
    strategies {
      network
      params
    }
  }
}
"""

res = snapshot_api(query)

<Response [400]>


KeyError: 'data'

In [32]:
res['proposals'][0]

{'id': '0x4a1aedbd9d22295f358dc4028b5a3f0a602bb5f1089dabdc2b63bf2bcce45834',
 'author': '0x0904Dac3347eA47d208F3Fd67402D039a3b99859',
 'created': 1662678865,
 'title': '[EP2.2.4] [Social] ENS Endaoment',
 'state': 'closed',
 'choices': ['For', 'Against', 'Abstain'],
 'scores': [2638265.201268065, 1379.7267441613128, 11.868487021979314],
 'scores_by_strategy': [[12237.15008958228, 2626028.0511784824],
  [0, 1379.7267441613128],
  [0, 11.868487021979314]],
 'scores_state': 'final',
 'scores_total': 2639656.796499248,
 'scores_updated': 1663110863,
 'strategies': [{'network': '1',
   'params': {'symbol': 'ENS (delegated)',
    'strategies': [{'name': 'erc20-votes',
      'params': {'symbol': 'ENS',
       'address': '0xC18360217D8F7Ab5e7c516566761Ea12Ce7F9D72',
       'decimals': 18}}]}},
  {'network': '1',
   'params': {'symbol': 'ENS',
    'address': '0xC18360217D8F7Ab5e7c516566761Ea12Ce7F9D72',
    'decimals': 18}}]}

In [33]:
res['proposals'][0]

{'id': '0x4a1aedbd9d22295f358dc4028b5a3f0a602bb5f1089dabdc2b63bf2bcce45834',
 'author': '0x0904Dac3347eA47d208F3Fd67402D039a3b99859',
 'created': 1662678865,
 'title': '[EP2.2.4] [Social] ENS Endaoment',
 'state': 'closed',
 'choices': ['For', 'Against', 'Abstain'],
 'scores': [2638265.201268065, 1379.7267441613128, 11.868487021979314],
 'scores_by_strategy': [[12237.15008958228, 2626028.0511784824],
  [0, 1379.7267441613128],
  [0, 11.868487021979314]],
 'scores_state': 'final',
 'scores_total': 2639656.796499248,
 'scores_updated': 1663110863,
 'strategies': [{'network': '1',
   'params': {'symbol': 'ENS (delegated)',
    'strategies': [{'name': 'erc20-votes',
      'params': {'symbol': 'ENS',
       'address': '0xC18360217D8F7Ab5e7c516566761Ea12Ce7F9D72',
       'decimals': 18}}]}},
  {'network': '1',
   'params': {'symbol': 'ENS',
    'address': '0xC18360217D8F7Ab5e7c516566761Ea12Ce7F9D72',
    'decimals': 18}}]}

In [37]:
res['proposals'][0]['scores'][0] == sum(res['proposals'][0]['scores_by_strategy'][0])

True

In [59]:
df = pd.DataFrame(res['proposals'][0]['choices'])
df.columns = ['choices']
df['id'] = res['proposals'][0]['id']

In [60]:
strategy_types = len(res['proposals'][0]['strategies'])

In [61]:
res['proposals'][0]['strategies'][0]

{'network': '1',
 'params': {'symbol': 'ENS (delegated)',
  'strategies': [{'name': 'erc20-votes',
    'params': {'symbol': 'ENS',
     'address': '0xC18360217D8F7Ab5e7c516566761Ea12Ce7F9D72',
     'decimals': 18}}]}}

In [62]:
for i in range(0,strategy_types):
    strategy_name = res['proposals'][0]['strategies'][i]['params']['symbol']
    strategy_name = 'strategy_' +  strategy_name
    df[strategy_name] = None

In [66]:
df

,choices,id,strategy_ENS (delegated),strategy_ENS
0,For,0x4a1aedbd9d22295f358dc4028b5a3f0a602bb5f1089d...,None,None
1,Against,0x4a1aedbd9d22295f358dc4028b5a3f0a602bb5f1089d...,None,None
2,Abstain,0x4a1aedbd9d22295f358dc4028b5a3f0a602bb5f1089d...,None,None


Another one: https://snapshot.org/#/cakevote.eth/proposal/bafkreiaaqkkd74p6npxmjohb63hzzulpmjfdaewqzs5ql2dvkrke6cj22i

In [64]:
query = """
query {
  proposals (
    where: { id: "bafkreiaaqkkd74p6npxmjohb63hzzulpmjfdaewqzs5ql2dvkrke6cj22i"}
  ) {
    id
    author
    created
    title
    state
    choices
    scores
    scores_by_strategy
    scores_state
    scores_total
    scores_updated
    strategies {
      network
      params
    }
  }
}
"""

res = snapshot_api(query)

<Response [200]>


In [65]:
res['proposals'][0]

{'id': 'bafkreiaaqkkd74p6npxmjohb63hzzulpmjfdaewqzs5ql2dvkrke6cj22i',
 'author': '0xaaFb9b50A560dA16519811C7cbc48DCd17320262',
 'created': 1665077667,
 'title': 'Index GamingShiba/ Volt (V2)',
 'state': 'closed',
 'choices': ['Make Pool Available for LP Token deposit', 'Remain as is.'],
 'scores': [35151.57636478451, 94808.5769715756],
 'scores_by_strategy': [[35151.57636478451], [94808.5769715756]],
 'scores_state': 'final',
 'scores_total': 129960.1533363601,
 'scores_updated': 1665111613,
 'strategies': [{'network': '56',
   'params': {'symbol': 'CAKE',
    'address': '0x0E09FaBB73Bd3Ade0a17ECC321fD13a19e81cE82',
    'decimals': 18}}]}

https://snapshot.org/#/aavegotchi.eth/proposal/0x5a2dc2a36b6d616a087c16cf2f4713e87abad28d9eb292a0e78a9872dec44665

In [202]:
query = """
query {
  proposals (
    where: { id: "0x5a2dc2a36b6d616a087c16cf2f4713e87abad28d9eb292a0e78a9872dec44665"}
  ) {
    id
    author
    created
    title
    state
    choices
    scores
    scores_by_strategy
    scores_state
    scores_total
    scores_updated
    strategies {
      network
      params
    }
  }
}
"""

res = snapshot_api(query)

<Response [200]>


In [203]:
res['proposals'][0]

{'id': '0x5a2dc2a36b6d616a087c16cf2f4713e87abad28d9eb292a0e78a9872dec44665',
 'author': '0x2848b9f2D4FaEBaA4838c41071684c70688B455d',
 'created': 1662176177,
 'title': 'Airdrop Gotchi Respec Potion',
 'state': 'closed',
 'choices': ['Yes, Airdrop  Potion', 'No', 'Abstaining'],
 'scores': [8637203.777684046, 1225116.9494226407, 776264.5436798697],
 'scores_by_strategy': [[276085.4077079329,
   5751131,
   37659.36997611606,
   2572328],
  [24562.171634978316, 802735, 9483.777787661942, 388336],
  [12566.216659130614, 617031, 1187.3270207391856, 145480]],
 'scores_state': 'final',
 'scores_total': 10638585.270786561,
 'scores_updated': 1663002012,
 'strategies': [{'network': '137',
   'params': {'limit': 200,
    'symbol': 'GHST',
    'strategy': {'name': 'aavegotchi',
     'params': {'decimals': 18,
      'tokenAddress': '0x385Eeac5cB85A38A9a07A70c73e0a3271CfB54A7',
      'stakingAddress': '0xA02d547512Bb90002807499F05495Fe9C4C3943f',
      'ghstUsdcAddress': '0x096c5ccb33cfc5732bcd1f31

In [204]:
res['proposals'][0]['strategies']

[{'network': '137',
  'params': {'limit': 200,
   'symbol': 'GHST',
   'strategy': {'name': 'aavegotchi',
    'params': {'decimals': 18,
     'tokenAddress': '0x385Eeac5cB85A38A9a07A70c73e0a3271CfB54A7',
     'stakingAddress': '0xA02d547512Bb90002807499F05495Fe9C4C3943f',
     'ghstUsdcAddress': '0x096c5ccb33cfc5732bcd1f3195c13dbefc4c82f4',
     'ghstWethAddress': '0xccb9d2100037f1253e6c1682adf7dc9944498aff',
     'ghstQuickAddress': '0x8b1fd78ad67c7da09b682c5392b65ca7caa101b9'}}}},
 {'network': '137',
  'params': {'limit': 500,
   'symbol': 'AGIP',
   'strategy': {'name': 'aavegotchi-agip',
    'params': {'tokenAddress': '0x86935f11c86623dec8a25696e1c19a8659cbf95d'}}}},
 {'network': '137',
  'params': {'limit': 500,
   'symbol': 'GHST',
   'strategy': {'name': 'erc20-balance-of',
    'params': {'address': '0x385Eeac5cB85A38A9a07A70c73e0a3271CfB54A7',
     'decimals': 18}}}},
 {'network': '137',
  'params': {'symbol': 'AGIP17',
   'strategy': {'name': 'aavegotchi-agip-17', 'params': {'

Experiment with a set of proposals

In [68]:
query = """
query {
  proposals (
    where: {space: "ens.eth"}
  ) {
    id
    author
    created
    title
    state
    choices
    votes
    scores
    scores_by_strategy
    scores_state
    scores_total
    scores_updated
    strategies {
      network
      params
    }
  }
}
"""

res = snapshot_api(query)

<Response [200]>


In [70]:
len(res['proposals'])

20

In [72]:
df = pd.DataFrame(res['proposals'])

In [80]:
len(df.iloc[0,:]['strategies'])

2

In [81]:
df['strategy_types'] = df['strategies'].apply(len)

In [148]:
def strategy_tags (strategy):
    tags = set()

    # dict keys
    for current_dict in strategy:
        tags = tags | set(current_dict['params'].keys())

    return tags

In [161]:
def strategy_classification (strategy_tags):
    output = []
    if {'address','symbol','decimals'}.issubset(strategy_tags) :
        output += ['balance-of']

    if set(['strategies','symbol']).issubset(strategy_tags) :
        output += ['delegated']
    
    return output

In [162]:
df['strategy_tags'] = df['strategies'].apply(strategy_tags)
df['strategy_classification'] = df['strategy_tags'].apply(strategy_classification)

In [163]:
df[['id','strategy_types','strategy_tags','strategy_classification']]

,id,strategy_types,strategy_tags,strategy_classification
0,0x4a1aedbd9d22295f358dc4028b5a3f0a602bb5f1089d...,2,"{address, strategies, symbol, decimals}","[balance-of, delegated]"
1,0x5c96e490f3e28d8269e8fc7e929491fb8fa5e4bd04d3...,2,"{address, strategies, symbol, decimals}","[balance-of, delegated]"
2,0x97265786d808280adc788e6744dd07afd3ff7e277652...,2,"{address, strategies, symbol, decimals}","[balance-of, delegated]"
3,0x46c7294aca8d70ae8213e8e8c6915697c7be1aab731f...,2,"{address, strategies, symbol, decimals}","[balance-of, delegated]"
4,0xa64ec8b446e509cb4b75092c4a714897790b70e2711f...,2,"{address, strategies, symbol, decimals}","[balance-of, delegated]"
5,0x3fa1a972b7d40510f5c1fb0d1c43e41960866a35db75...,2,"{address, strategies, symbol, decimals}","[balance-of, delegated]"
6,0x1b1ed675d08dd689148133c5be076e78997500d19698...,2,"{address, strategies, symbol, decimals}","[balance-of, delegated]"
7,0x056dd964a9fb8da60cfc46d82c3770b09b22212c030c...,2,"{address, strategies, symbol, decimals}","[balance-of, delegated]"
8,0x3742c2d352f92260e2c827767266a88fd214d2cd309e...,2,"{address, strategies, symbol, decimals}","[balance-of, delegated]"
9,0xe07284156fb063d5fba6b9fed50cc74fad36ea02c2ed...,2,"{address, strategies, symbol, decimals}","[balance-of, delegated]"


Operation

In [226]:
q = get_query("snapshot_proposals2.gql", do_gql=False)

proposal = await gql_all(query = q, 
                        field = 'proposals',
                        #limit = 1,
                        save='proposals_20221007.json',
                        save_interval=10
)

1 1000
2 2000
3 3000
4 4000
5 5000
6 6000
7 7000
8 8000
9 9000
10 10000
Saved.
11 11000
12 12000
13 13000
14 14000
15 15000
16 16000
17 17000
18 18000
19 19000
20 20000
Saved.
21 21000
22 22000
23 23000
24 24000
25 25000
26 26000
27 27000
28 28000
29 29000
30 30000
Saved.
31 31000
32 32000
33 33000
34 34000
35 35000
36 36000
37 37000
38 38000
39 39000
40 40000
Saved.
41 41000
42 42000
43 43000
44 44000
45 45000
46 46000
47 47000
48 48000
49 49000
50 50000
Saved.
51 51000
52 52000
53 53000
54 54000
55 55000
56 56000
57 57000
58 58000
59 59000
60 60000
Saved.
61 61000
62 62000
63 63000
64 64000
65 65000
66 66000
67 67000
68 68000
69 69000
70 69801
Saved.
**I am done fetching!**
Saved.


In [227]:
with open("../4_power/data/proposals_20221007.json") as file:
    proposals = json.load(file)

proposals = pd.DataFrame(proposals)
proposals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69801 entries, 0 to 69800
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  69801 non-null  object 
 1   created             69801 non-null  int64  
 2   space               69801 non-null  object 
 3   network             69801 non-null  object 
 4   symbol              69801 non-null  object 
 5   type                69801 non-null  object 
 6   strategies          69801 non-null  object 
 7   title               69801 non-null  object 
 8   body                69801 non-null  object 
 9   choices             69801 non-null  object 
 10  start               69801 non-null  int64  
 11  end                 69801 non-null  int64  
 12  quorum              69801 non-null  float64
 13  snapshot            69801 non-null  object 
 14  state               69801 non-null  object 
 15  link                69801 non-null  object 
 16  app 

In [229]:
proposals.votes.describe()

count     69801.000000
mean         93.069727
std        1222.786352
min           0.000000
25%           1.000000
50%           7.000000
75%          33.000000
max      220917.000000
Name: votes, dtype: float64

In [231]:
proposals = proposals[proposals.votes>0]

In [232]:
proposals.votes.describe()

count     56993.000000
mean        113.985226
std        1352.347972
min           1.000000
25%           3.000000
50%          12.000000
75%          45.000000
max      220917.000000
Name: votes, dtype: float64

In [235]:
proposals[['id','strategies']].head()

,id,strategies
0,QmSXpyHBrkjzjmGpdYxKvYoRAMvdJNp7rrnyvJqTdtpayZ,"[{'params': {'symbol': 'BAL', 'address': '0xba..."
1,QmXYVj4GBu16GfuTLouxQSvbB8w3G2EAvYq9TAxX38ruKb,"[{'params': {'symbol': 'BAL', 'address': '0xba..."
2,QmboDWvsXYXPZb44arPVoMa3LHuv74YsGZ3cSoG6uj4oZe,"[{'params': {'symbol': 'BAL', 'address': '0xba..."
3,QmeWmvdFCTq5tnFHqgFmAVJDCccTYjg1zWb9cfUByg7zbz,"[{'params': {'symbol': 'BAL', 'address': '0xba..."
4,QmUadcUgd3jAhJbofgrnKNMfeuGFvMmybhCkttW4gGfYYJ,"[{'params': {'symbol': 'BAL', 'address': '0xba..."


In [236]:
proposals['strategy_types'] = proposals['strategies'].apply(len)
proposals['strategy_tags'] = proposals['strategies'].apply(strategy_tags)
#proposals['strategy_classification'] = proposals['strategy_tags'].apply(strategy_classification)

In [239]:
proposals.strategy_tags.value_counts().head(20)

{address, decimals, symbol}                                          26840
{address, symbol}                                                     4940
{}                                                                    2879
{symbol}                                                              2347
{address, decimals, methodABI, symbol}                                2052
{address, chefAddresses}                                              1987
{graphs, strategies, symbol, multiplier, decimals, address, name}     1530
{address, strategies, symbol, decimals}                               1251
{address, decimals, network, symbol}                                   839
{address, tokenId, decimals, symbol}                                   684
{addresses, symbol}                                                    674
{address, decimals, minBalance, symbol}                                584
{address, args, symbol, decimals, methodABI}                           453
{address, strategies, sym

In [263]:
proposals.strategy_types.value_counts().head(20)

1    42239
2     8093
3     2242
6     1745
4     1407
7      601
5      554
8      112
Name: strategy_types, dtype: int64

In [278]:
proposals['strategy_tags_str'] = proposals['strategy_tags'].apply(str)

In [279]:
proposals[['strategy_types','strategy_tags_str','id']].groupby(['strategy_types','strategy_tags_str']).count().reset_index().sort_values(by='id',ascending=False).head(10)

,strategy_types,strategy_tags_str,id
33,1,"{'address', 'decimals', 'symbol'}",22683
62,1,"{'address', 'symbol'}",4556
0,1,set(),2879
240,2,"{'address', 'symbol', 'decimals'}",2211
150,1,{'symbol'},2153
16,1,"{'address', 'chefAddresses'}",1987
27,1,"{'address', 'decimals', 'methodABI', 'symbol'}",1401
640,6,"{'graphs', 'strategies', 'symbol', 'multiplier...",1397
218,2,"{'address', 'strategies', 'symbol', 'decimals'}",917
29,1,"{'address', 'decimals', 'network', 'symbol'}",778


In [253]:
proposals.loc[proposals.strategy_tags == set(),('id','space','votes','strategies')]

,id,space,votes,strategies
41999,0x2b038886a35bdfa6abd4fafccde2d6ff9d82e264688f...,{'id': 'happyguy.eth'},1,[{'params': {}}]
42002,0x2997f45864b760ee662369024e324b710b207d3d2b0e...,{'id': 'sherlin.eth'},2,[{'params': {}}]
42139,0x8279fbf102c6d9b022a7a608a43794d54eea87ef4e40...,{'id': 'jray.eth'},1,[{'params': {}}]
42140,0x8c07b047a58c1d4b30743085f018e8965d859bb6fcc7...,{'id': 'jray.eth'},1,[{'params': {}}]
42185,0x9995d85eb2ec5e4c3b52fd6349eaffa9a8187684a01c...,{'id': 'petroleumdefi.eth'},30,[{'params': {}}]
...,...,...,...,...
69304,bafkreidghfdm3u3biqjroqedk2iag5wybhpxlvicgupj2...,{'id': 'aggris88.eth'},1,[{'params': {}}]
69305,bafkreih2tgrmd2f47ogoy5bxjzfca63ur5kdhpmchj6x2...,{'id': 'aggris88.eth'},4,[{'params': {}}]
69477,0x572b08f72485a8490ae9d85a793ee6c88ca34b7f567b...,{'id': '0xflowers.eth'},319,[{'params': {}}]
69521,0x0e0b2654fd453c2ac5827beabdbc5d877b6564df7c30...,{'id': 'jigglyjams.eth'},1,[{'params': {}}]


In [259]:
proposals.loc[proposals.index==69477,'id'].values

array(['0x572b08f72485a8490ae9d85a793ee6c88ca34b7f567b9a23e1fff5b0933f292f'],
      dtype=object)

In [260]:
query = """
query {
  proposals (
    where: {id: "0x572b08f72485a8490ae9d85a793ee6c88ca34b7f567b9a23e1fff5b0933f292f"}
  ) {
    id
    author
    created
    title
    state
    choices
    votes
    scores
    scores_by_strategy
    scores_state
    scores_total
    scores_updated
    strategies {
      network
      params
    }
  }
}
"""

res = snapshot_api(query)

<Response [200]>


In [262]:
res['proposals']

[{'id': '0x572b08f72485a8490ae9d85a793ee6c88ca34b7f567b9a23e1fff5b0933f292f',
  'author': '0x288db71982541c18765dC6b018BD264649f04340',
  'created': 1664901191,
  'title': 'Wen airdrop sir?',
  'state': 'active',
  'choices': ['why not', 'not interesting'],
  'votes': 329,
  'scores': [298, 31],
  'scores_by_strategy': [[298], [31]],
  'scores_state': 'pending',
  'scores_total': 329,
  'scores_updated': 1665147683,
  'strategies': [{'network': '42161', 'params': {}}]}]